# Table of Contents
* [Feature Requests](#feature-requests)
* [Questions](#questions)
* [Bugs](#bugs)

In [1]:
# Standard library imports
import os

# Third-party library imports
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the GameSim API key
# .env file format:
# GAMESIM_API_KEY="<api key>"
api_key = os.getenv('GAMESIM_API_KEY')

base_gamesim_url = 'https://api.ncaagamesim.com'

# Usage is explained in "Potential Bugs/Feature Requests" Number 1.
postman_user_agent = 'PostmanRuntime/7.33.0'

# Set default headers for requests.
headers = {
    'Authorization': api_key,
    'User-Agent': postman_user_agent
}

# Questions <a class="anchor" id="questions"></a>

### 1. ncaaScheduleId/nflScheduleId

The request data for a simulation post is as follows:

<br/>

```python
request_data = {
    "hometeam": "homeTeam",
    "homeyear": "homeYear",
    "awayteam": "awayTeam",
    "awayyear": "awayYear",
    "includeStats": True,
    "ncaaScheduleId": 0,
    "nflScheduleId": 0,
    "numberOfSimulations": 0
}
```
<br/>
Curious what 'ncaaScheduleId'/'nflScheduleId' represent or control? With some basic testing I was able to pass various integers into those fields and couldn't see any difference in the returned data.

<br/>

# Feature Requests <a class="anchor" id="feature-requests"></a>

### 1. API Usage Endpoint/Field

Currently there doesn't appear to be a way to see how many calls you have made to the API in the given billing timeframe. This would be very helpful, weather it's apart of each response (prefered in my opinion) or a seperate endput.

For example, my personal preference would be that a Simulate call would return a structure somewhat similar to this. Could be even more basic, really only need callsMade OR callsRemaining. Preferably (again, in my opinion) callsRemainig, especially if you plan to have different tiers of API subscriptions where some have a higher number of alloted calls.

<br/>


```json
{
  "rateLimit": [
    {
      "callsMade": 324,
      "callsRemaining": 176,
      "daysRemainingCurrentCycle": 11
    }
  ],
  "data": [
    {
      "hometeam": "Illinois",
      "homename": "",
      "homeyear": 2024,
      "awayteam": "Wisconsin",
      "awayname": "",
      "awayyear": 2024,
      "homescore": 17,
      "awayscore": 13
    }
  ]
}
```

<br/>

### 2. Home Field Advantage and Show Play-by-Play fields

Would love to have access to these options from the API.
<br/>
```python
request_data = {
    "hometeam": "Illinois",
    "homeyear": "2024",
    "awayteam": "Iowa",
    "awayyear": "2024",
    "includeStats": True,
    "ncaaScheduleId": 0,
    "nflScheduleId": 0,
    "numberOfSimulations": 0,
    "homeFieldAdvantage": 1,
    "showPlayByPlay": 1
}

sport = 'cfb'

simulate_response = requests.post(base_gamesim_url + f'/gamesim/simulate/cfb', headers=headers, json=request_data)
```

<br/>

### 3. Predictions Endpoint {dtgame} parameter clarification

Took me a bit of time to figure out the required datetime format - although I should have gotten there sooner it might be nice to have that format explicitly shown in the documentation as opposed to just "string($date-time). Not a biggie at all.

Regardless of that point, one of my feature requests was going to be the ability to get all predictions for a league based on only the date, and not the time. It appears at least today (10/20) that is actually the case, and the time portion of the dtgame parameter does not matter. Personally hope that this is intended, as otherwise gametimes would be needed to get any predicitons. Wouldn't be impossible, but at least for me would be less "natural". This way for any given week/period of time I can make one call per day and not have to loop through times. 

In [2]:
sport = 'cfb'

# With Time
year, month, day = "2023", "10", "21"
hour, minute, second = "04", "30", "00"
dtgame = f"{year}-{month}-{day}%20{hour}%3A{minute}%3A{second}"

prediction_response_with_time = requests.get(base_gamesim_url + f'/gamesim/predictions/{sport}/{dtgame}', headers=headers)
print(f"prediction responses without time: {len(prediction_response_with_time.json())}")


# Without Time
hour, minute, second = "00", "00", "00"
dtgame = f"{year}-{month}-{day}%20{hour}%3A{minute}%3A{second}"

prediction_response_without_time = requests.get(base_gamesim_url + f'/gamesim/predictions/{sport}/{dtgame}', headers=headers)
print(f"prediction responses without time: {len(prediction_response_without_time.json())}")
print(f"With & Without time equivalent?: {prediction_response_with_time.json() == prediction_response_without_time.json()}")

prediction responses without time: 101
prediction responses without time: 101
With & Without time equivalent?: True


### 4. Predictions by Week
Along those same lines, I think it would be awesome to be able to query predicitons based on week number and not date. Would only really make sense for NFL/CFB, but if it was possible to call:
<br/>
<br/>
```python
requests.get(base_gamesim_url + f'/gamesim/predictions/nfl/week7', headers=headers)
```
<br/>
and get all 13 predictions that would be great. Super low priority especially if the game time doesn't matter in the query.

<br/><br/>

# Bugs <a class="anchor" id="bugs"></a>

### 1. Simulate request with "includeStats" set to false throws a 500 Internal Server Error.

In [3]:
headers = {
    'Authorization': api_key,
    'User-Agent': postman_user_agent
}

request_data = {
    "hometeam": "Illinois",
    "homeyear": "2024",
    "awayteam": "Wisconsin",
    "awayyear": "2024",
    "includeStats": True,
    "ncaaScheduleId": 0,
    "nflScheduleId": 0,
    "numberOfSimulations": 0

}

sport = 'cfb'

simulate_response = requests.post(base_gamesim_url + f'/gamesim/simulate/{sport}', headers=headers, json=request_data)
print(f"includeStats True: {simulate_response.status_code} {simulate_response.reason}")

# Set includeStats to False
request_data['includeStats'] = "False"

simulate_response = requests.post(base_gamesim_url + f'/gamesim/simulate/{sport}', headers=headers, json=request_data)
print(f"includeStats False: {simulate_response.status_code} {simulate_response.reason}")

includeStats True: 200 OK
includeStats False: 400 Bad Request


<br/><br/>

### 2. Only certain User-Agents have Access Permisions

This isn't even necessarily a bug, just thought it was worth pointing out.


Without Set User-Agent:

In [4]:
headers = {
    'Authorization': api_key
}

response = requests.get('https://api.ncaagamesim.com/gamesim/ping', headers=headers)
print(response.status_code, response.reason)

403 ModSecurity Action


<br/>

We can use [hhtpbin](https://httpbin.org/) to easily see the auto-generated headers of our request.

In [5]:
request_with_all_headers = requests.get('http://httpbin.org/headers') # Note no headers are included in this request.
print(f"Auto-Generated User-Agent: {request_with_all_headers.json()['headers']['User-Agent']}")

Auto-Generated User-Agent: python-requests/2.28.1


<br/>

Python automatically assigned the User-Agent to "python-requests/2.28.1", which was denied access. Since we know this request has worked in Postman before, let's try setting 'User-Agent' to the Postman value.

### With Set User-Agent:

In [6]:
headers = {
    'Authorization': api_key,
    'User-Agent': 'PostmanRuntime/7.33.0'
}

response = requests.get('https://api.ncaagamesim.com/gamesim/ping', headers=headers)
print(response.status_code, response.reason)

200 OK


Assume this is intended on some level, would be curious if possible to expand the User-Agents with permissions, especially because it is easy to spoof the User-Agent. Not a huge deal at all. 


Tested with the following User-Agents as well:

|          User-Agent      |      Response Status       |
| :-----------------------:| :------------------------: |
|  PostmanRuntime/7.33.0   |    200 OK                  |
|  Boomi Http Transport*   |    200 OK                  |  
|  python-requests/2.28.1  |    403 ModSecurity Action  |
|  python-requests/2.30.0  |    403 ModSecurity Action  |

*User-Agent auto-generated by Boomi